In [2]:
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path
DATA_PATH = Path() / "../data"
#DATA_PATH = Path('/content/gdrive/My Drive/Colab Notebooks/VC_data_Cleaning_With_LLM/data')
DATA_PATH.mkdir(parents=True,exist_ok=True)

def load_data(filename, data_path=DATA_PATH,encoding='ISO-8859-1'):
    csv_path = data_path / filename
    return pd.read_csv(csv_path,encoding=encoding)

def save_data(data, filename, data_path=DATA_PATH,encoding='ISO-8859-1'):
    csv_path = data_path / filename
    data.to_csv(csv_path, index=False,encoding='ISO-8859-1')

PLOT_PATH = Path() / "../plot"
#PLOT_PATH = Path('/content/gdrive/My Drive/Colab Notebooks/VC_data_Cleaning_With_LLM/plot')
PLOT_PATH.mkdir(parents=True,exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300, transparent=True):
    path = PLOT_PATH / f"{fig_id}.{fig_extension}"
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution, transparent=transparent)

RESULT_PATH = Path() / "../result"
#PLOT_PATH = Path('/content/gdrive/My Drive/Colab Notebooks/VC_data_Cleaning_With_LLM/plot')
RESULT_PATH.mkdir(parents=True,exist_ok=True)
def save_result(data, filename, data_path=RESULT_PATH):
    csv_path = data_path / filename
    data.to_csv(csv_path, index=False, encoding='utf-8-sig')

def save_excel(data, filename, data_path=RESULT_PATH):
    csv_path = data_path / filename
    data.to_excel(csv_path, index=False)


In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from kor.extraction import create_extraction_chain
from kor.nodes import Object,Text, Number
# Import the openai module
import openai

In [4]:
import json
def json_dump(json_object):
    json_formatted_str = json.dumps(json_object, indent=2,ensure_ascii= False)
    print(json_formatted_str)

In [5]:
from dotenv import load_dotenv
import os

load_dotenv()
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')

In [6]:
from typing import Optional

from langchain_core.pydantic_v1 import BaseModel, Field


class Person(BaseModel):
    """Information about a person."""

    # ^ Doc-string for the entity Person.
    # This doc-string is sent to the LLM as the description of the schema Person,
    # and it can help to improve extraction results.

    # Note that:
    # 1. Each field is an `optional` -- this allows the model to decline to extract it!
    # 2. Each field has a `description` -- this description is used by the LLM.
    # Having a good description can help improve extraction results.
    role: Optional[str] = Field(
        ..., description="The role or title of this person. "
                                )
    company: Optional[str] = Field(
        ..., description="The company this person is working in."
    )
    ai_related: Optional[bool] = Field(
        ..., description="Whether this person is related to AI."
    )

In [7]:
from typing import Optional

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

# Define a custom prompt to provide instructions and any additional context.
# 1) You can add examples into the prompt template to improve extraction quality
# 2) Introduce additional parameters to take context into account (e.g., include metadata
#    about the document from which the text was extracted.)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "You are an expert extraction algorithm. "
            "Only extract relevant information from the text. "
            "If you do not know the value of an attribute asked to extract"
            "return null for the attribute's value."

            "The text we input is a person's biography. "
            "You should treat every input text as a separate entity."

            "For each person's biography, you should consider:"
            "If this person has several roles, we should only annotate one."
            "If this person is a AI company founder or co-founder, annotate him as a co-founder."
            "If this person is a researcher in AI feild, annotate this person as a researcher."
            "If you can't find any role, try to conclude the role from the context, if you still can't find any role, please annotate the role as NA."

            "For all the biographies with roles as NA, please read the context and try to judge if the biography is from a enterprise account's biography. "
            "If it is, please annotate the role as 'Y'. If you can't judge, keep the role annotation as 'NA'."

            "when you extract company name, please extract the full name of the company and exclude any other punctuation mark such as @."
            "If there is no company name, please annotate the company name as NA."

            "If this person is related to AI, or anything about AI, please annotate the AI_related as true, otherwise annotate it as false."

            "The most important thing is you must extract exactly one combo of information for each input text! not more than one combo of information or less!"

        ),
        # Please see the how-to about improving performance with
        # reference examples.
        # MessagesPlaceholder('examples'),
        ("human", "{text}"),
    ]
)

In [8]:
from typing import List, Optional
class Data(BaseModel):
    """Extracted data about people."""

    # Creates a model so that we can extract multiple entities.
    people: List[Person]

In [9]:
import uuid
from typing import Dict, List, TypedDict
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.messages import (
    AIMessage,
    BaseMessage,
    HumanMessage,
    SystemMessage,
    ToolMessage,
)

class Example(TypedDict):
    """A representation of an example consisting of text input and expected tool calls.

    For extraction, the tool calls are represented as instances of pydantic model.
    """

    input: str  # This is the example text
    tool_calls: List[BaseModel]  # Instances of pydantic model that should be extracted


In [10]:
def tool_example_to_messages(example: Example) -> List[BaseMessage]:
    """Convert an example into a list of messages that can be fed into an LLM.

    This code is an adapter that converts our example to a list of messages
    that can be fed into a chat model.

    The list of messages per example corresponds to:

    1) HumanMessage: contains the content from which content should be extracted.
    2) AIMessage: contains the extracted information from the model
    3) ToolMessage: contains confirmation to the model that the model requested a tool correctly.

    The ToolMessage is required because some of the chat models are hyper-optimized for agents
    rather than for an extraction use case.
    """
    messages: List[BaseMessage] = [HumanMessage(content=example["input"])]
    openai_tool_calls = []
    for tool_call in example["tool_calls"]:
        openai_tool_calls.append(
            {
                "id": str(uuid.uuid4()),
                "type": "function",
                "function": {
                    # The name of the function right now corresponds
                    # to the name of the pydantic model
                    # This is implicit in the API right now,
                    # and will be improved over time.
                    "name": tool_call.__class__.__name__,
                    "arguments": tool_call.json(),
                },
            }
        )
    messages.append(
        AIMessage(content="", additional_kwargs={"tool_calls": openai_tool_calls})
    )
    tool_outputs = example.get("tool_outputs") or [
        "You have correctly called this tool."
    ] * len(openai_tool_calls)
    for output, tool_call in zip(tool_outputs, openai_tool_calls):
        messages.append(ToolMessage(content=output, tool_call_id=tool_call["id"]))
    return messages

In [11]:
examples = [
    (
        "Research Scientist at Google DeepMind. I tweet about deep learning (research + software), music, generative models (personal account).",
        Person(role="Research Scientist", company="Google DeepMind", interest="deep learning, music, and generative models", ai_related=True),
    ),
    (
        "I make videos.\nSkill &gt; Destiny.\nvi / vim",
        Person(role=None, company=None, interest=None, ai_related=False),
    ),
    (
        "ML Researcher, co-leading Superalignment @OpenAI. Optimizing for a post-AGI future where humanity flourishes.",
        Person(role="ML Researcher", company="OpenAI", interest="AGI", ai_related=True),
    )
]


messages = []

for text, tool_call in examples:
    messages.extend(
        tool_example_to_messages({"input": text, "tool_calls": [tool_call]})
    )

In [12]:
llm = ChatOpenAI(
    model_name = 'gpt-4-turbo-preview',
    temperature = 0,
    openai_api_key = OPENAI_API_KEY
)

In [13]:
runnable = prompt | llm.with_structured_output(
    schema=Person,
    method="function_calling",
    include_raw=False,
)

c:\Users\TYS\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\beta_decorator.py:87: LangChainBetaWarning: The function `with_structured_output` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


In [14]:
# Apply the function to remove illegal XML characters to every string in the DataFrame
# and save directly to an Excel file without the intermediate cleaning step.
def clean_and_save_dataset(dataframe):
    # Define the function to remove illegal XML characters
    def remove_illegal_xml_characters(s):
        if isinstance(s, str):
            return ''.join(char for char in s if ord(char) >= 32 or char in '\t\n\r')
        else:
            return s
    
    # Apply the cleaning function across the DataFrame
    cleaned_df = dataframe.applymap(remove_illegal_xml_characters)
    return cleaned_df

In [15]:
df_new = load_data('twitter1.csv')
# Drop NaN values from the 'bio' column
df_new = df_new.dropna(subset=['bio'])
# Sort the DataFrame by 'tweetsCount' in descending order
#df_new = df_new.sort_values(by='followersCount', ascending=False)

In [16]:
bio_text = df_new['bio'].tolist()

In [17]:
df_intermediate = pd.DataFrame()
for idx, text in enumerate(bio_text):
    extracted_info = runnable.invoke({"text": text, "examples": messages})
    #bio_data = [{"role": person.role, "company": person.company, "ai_related": person.ai_related} for person in extracted_info.people]
    bio_data = {"role": extracted_info.role, "company": extracted_info.company, "ai_related": extracted_info.ai_related,"row_index":idx}
    bio_df_new = pd.DataFrame(bio_data, index=[0])
    # extracted_info_list = pd.concat([extracted_info_list, bio_df_new])
    print(bio_df_new.shape[0])
    df_intermediate = pd.concat([df_intermediate, bio_df_new])

ValueError: If using all scalar values, you must pass an index

In [18]:
bio_data

{'role': 'NA', 'company': 'NA', 'ai_related': True}

In [ ]:
df_intermediate

In [18]:
df_new = df_new.reset_index(drop=True)
extracted_info_list = df_intermediate.reset_index(drop=True)  # Assuming it's a DataFrame
result = pd.concat([df_new, extracted_info_list], axis=1)

In [ ]:
result

In [20]:
save_excel(result, 'result_temp1.xlsx')

In [ ]:
df_new = load_data('twitter2.csv')
# Drop NaN values from the 'bio' column
df_new = df_new.dropna(subset=['bio'])
# Sort the DataFrame by 'tweetsCount' in descending order
df_new = df_new.sort_values(by='followersCount', ascending=False)
bio_text = df_new['bio'].tolist()
df_intermediate = pd.DataFrame()
counter = 0
for text in bio_text:
    extracted_info = runnable.invoke({"text": text, "examples": messages})
    bio_data = [{"role": person.role, "company": person.company, "ai_related": person.ai_related} for person in extracted_info.people]
    bio_df_new = pd.DataFrame(bio_data)
    counter += 1
    print(counter)
    print(bio_df_new.shape[0])
    # extracted_info_list = pd.concat([extracted_info_list, bio_df_new])
    df_intermediate = pd.concat([df_intermediate, bio_df_new])
df_new = df_new.reset_index(drop=True)
extracted_info_list = df_intermediate.reset_index(drop=True)  # Assuming it's a DataFrame
result = pd.concat([df_new, extracted_info_list], axis=1)
save_excel(result, 'result_temp2.xlsx')

In [ ]:
df_new = load_data('twitter3.csv')
# Drop NaN values from the 'bio' column
df_new = df_new.dropna(subset=['bio'])
# Sort the DataFrame by 'tweetsCount' in descending order
df_new = df_new.sort_values(by='followersCount', ascending=False)
bio_text = df_new['bio'].tolist()
df_intermediate = pd.DataFrame()
counter = 0
for text in bio_text:
    extracted_info = runnable.invoke({"text": text, "examples": messages})
    bio_data = [{"role": person.role, "company": person.company, "ai_related": person.ai_related} for person in extracted_info.people]
    bio_df_new = pd.DataFrame(bio_data)
    counter+=1
    print("round: " , counter)
    print(bio_df_new.shape[0])
    # extracted_info_list = pd.concat([extracted_info_list, bio_df_new])
    df_intermediate = pd.concat([df_intermediate, bio_df_new])
df_new = df_new.reset_index(drop=True)
extracted_info_list = df_intermediate.reset_index(drop=True)  # Assuming it's a DataFrame
result = pd.concat([df_new, extracted_info_list], axis=1)
save_excel(result, 'result_temp3.xlsx')

In [ ]:
df_new = load_data('twitter4.csv')
# Drop NaN values from the 'bio' column
df_new = df_new.dropna(subset=['bio'])
# Sort the DataFrame by 'tweetsCount' in descending order
df_new = df_new.sort_values(by='followersCount', ascending=False)
bio_text = df_new['bio'].tolist()
df_intermediate = pd.DataFrame()
counter = 0
for text in bio_text:
    extracted_info = runnable.invoke({"text": text, "examples": messages})
    bio_data = [{"role": person.role, "company": person.company, "ai_related": person.ai_related} for person in extracted_info.people]
    bio_df_new = pd.DataFrame(bio_data)
    print("round: " , counter)
    print(bio_df_new.shape[0])
    # extracted_info_list = pd.concat([extracted_info_list, bio_df_new])
    df_intermediate = pd.concat([df_intermediate, bio_df_new])
df_new = df_new.reset_index(drop=True)
extracted_info_list = df_intermediate.reset_index(drop=True)  # Assuming it's a DataFrame
result = pd.concat([df_new, extracted_info_list], axis=1)
save_excel(result, 'result_temp4.xlsx')

In [25]:
df_new = load_data('twitter5.csv')
# Drop NaN values from the 'bio' column
df_new = df_new.dropna(subset=['bio'])
# Sort the DataFrame by 'tweetsCount' in descending order
df_new = df_new.sort_values(by='followersCount', ascending=False)
bio_text = df_new['bio'].tolist()
df_intermediate = pd.DataFrame()
counter = 0
for text in bio_text:
    extracted_info = runnable.invoke({"text": text, "examples": messages})
    bio_data = [{"role": person.role, "company": person.company, "ai_related": person.ai_related} for person in extracted_info.people]
    bio_df_new = pd.DataFrame(bio_data)
    print("round: " , counter)
    print(bio_df_new.shape[0])
    # extracted_info_list = pd.concat([extracted_info_list, bio_df_new])
    df_intermediate = pd.concat([df_intermediate, bio_df_new])
df_new = df_new.reset_index(drop=True)
extracted_info_list = df_intermediate.reset_index(drop=True)  # Assuming it's a DataFrame
result = pd.concat([df_new, extracted_info_list], axis=1)
save_excel(result, 'result_temp5.xlsx')

round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
round:  0
1
roun

In [27]:
extracted_info

Data(people=[Person(role='NA', company='NA', ai_related=False)])

In [ ]:
df_new = load_data('twitter6.csv')
# Drop NaN values from the 'bio' column
df_new = df_new.dropna(subset=['bio'])
# Sort the DataFrame by 'tweetsCount' in descending order
df_new = df_new.sort_values(by='followersCount', ascending=False)
bio_text = df_new['bio'].tolist()
df_intermediate = pd.DataFrame()
counter = 0
for text in bio_text:
    extracted_info = runnable.invoke({"text": text, "examples": messages})
    bio_data = [{"role": person.role, "company": person.company, "ai_related": person.ai_related} for person in extracted_info.people]
    bio_df_new = pd.DataFrame(bio_data)
    print("round: " + counter)
    print(bio_df_new.shape[0])
    # extracted_info_list = pd.concat([extracted_info_list, bio_df_new])
    df_intermediate = pd.concat([df_intermediate, bio_df_new])
df_new = df_new.reset_index(drop=True)
extracted_info_list = df_intermediate.reset_index(drop=True)  # Assuming it's a DataFrame
result = pd.concat([df_new, extracted_info_list], axis=1)
save_excel(result, 'result_temp6.xlsx')

In [ ]:
df_new = load_data('twitter7.csv')
# Drop NaN values from the 'bio' column
df_new = df_new.dropna(subset=['bio'])
# Sort the DataFrame by 'tweetsCount' in descending order
df_new = df_new.sort_values(by='followersCount', ascending=False)
bio_text = df_new['bio'].tolist()
df_intermediate = pd.DataFrame()
counter = 0
for text in bio_text:
    extracted_info = runnable.invoke({"text": text, "examples": messages})
    bio_data = [{"role": person.role, "company": person.company, "ai_related": person.ai_related} for person in extracted_info.people]
    bio_df_new = pd.DataFrame(bio_data)
    print("round: " + counter)
    print(bio_df_new.shape[0])
    # extracted_info_list = pd.concat([extracted_info_list, bio_df_new])
    df_intermediate = pd.concat([df_intermediate, bio_df_new])
df_new = df_new.reset_index(drop=True)
extracted_info_list = df_intermediate.reset_index(drop=True)  # Assuming it's a DataFrame
result = pd.concat([df_new, extracted_info_list], axis=1)
save_excel(result, 'result_temp7.xlsx')

In [ ]:
df_new = load_data('twitter8.csv')
# Drop NaN values from the 'bio' column
df_new = df_new.dropna(subset=['bio'])
# Sort the DataFrame by 'tweetsCount' in descending order
df_new = df_new.sort_values(by='followersCount', ascending=False)
bio_text = df_new['bio'].tolist()
df_intermediate = pd.DataFrame()
counter = 0
for text in bio_text:
    extracted_info = runnable.invoke({"text": text, "examples": messages})
    bio_data = [{"role": person.role, "company": person.company, "ai_related": person.ai_related} for person in extracted_info.people]
    bio_df_new = pd.DataFrame(bio_data)
    print("round: " + counter)
    print(bio_df_new.shape[0])
    # extracted_info_list = pd.concat([extracted_info_list, bio_df_new])
    df_intermediate = pd.concat([df_intermediate, bio_df_new])
df_new = df_new.reset_index(drop=True)
extracted_info_list = df_intermediate.reset_index(drop=True)  # Assuming it's a DataFrame
result = pd.concat([df_new, extracted_info_list], axis=1)
save_excel(result, 'result_temp8.xlsx')

In [ ]:
df_new = load_data('twitter9.csv')
# Drop NaN values from the 'bio' column
df_new = df_new.dropna(subset=['bio'])
# Sort the DataFrame by 'tweetsCount' in descending order
df_new = df_new.sort_values(by='followersCount', ascending=False)
bio_text = df_new['bio'].tolist()
df_intermediate = pd.DataFrame()
counter = 0
for text in bio_text:
    extracted_info = runnable.invoke({"text": text, "examples": messages})
    bio_data = [{"role": person.role, "company": person.company, "ai_related": person.ai_related} for person in extracted_info.people]
    bio_df_new = pd.DataFrame(bio_data)
    print("round: " + counter)
    print(bio_df_new.shape[0])
    # extracted_info_list = pd.concat([extracted_info_list, bio_df_new])
    df_intermediate = pd.concat([df_intermediate, bio_df_new])
df_new = df_new.reset_index(drop=True)
extracted_info_list = df_intermediate.reset_index(drop=True)  # Assuming it's a DataFrame
result = pd.concat([df_new, extracted_info_list], axis=1)
save_excel(result, 'result_temp9.xlsx')

In [ ]:
df_new = load_data('twitter10.csv')
# Drop NaN values from the 'bio' column
df_new = df_new.dropna(subset=['bio'])
# Sort the DataFrame by 'tweetsCount' in descending order
df_new = df_new.sort_values(by='followersCount', ascending=False)
bio_text = df_new['bio'].tolist()
df_intermediate = pd.DataFrame()
counter = 0
for text in bio_text:
    extracted_info = runnable.invoke({"text": text, "examples": messages})
    bio_data = [{"role": person.role, "company": person.company, "ai_related": person.ai_related} for person in extracted_info.people]
    bio_df_new = pd.DataFrame(bio_data)
    print("round: " + counter)
    print(bio_df_new.shape[0])
    # extracted_info_list = pd.concat([extracted_info_list, bio_df_new])
    df_intermediate = pd.concat([df_intermediate, bio_df_new])
df_new = df_new.reset_index(drop=True)
extracted_info_list = df_intermediate.reset_index(drop=True)  # Assuming it's a DataFrame
result = pd.concat([df_new, extracted_info_list], axis=1)
save_excel(result, 'result_temp10.xlsx')

In [ ]:
df_new = load_data('twitter11.csv')
# Drop NaN values from the 'bio' column
df_new = df_new.dropna(subset=['bio'])
# Sort the DataFrame by 'tweetsCount' in descending order
df_new = df_new.sort_values(by='followersCount', ascending=False)
bio_text = df_new['bio'].tolist()
df_intermediate = pd.DataFrame()
counter = 0
for text in bio_text:
    extracted_info = runnable.invoke({"text": text, "examples": messages})
    bio_data = [{"role": person.role, "company": person.company, "ai_related": person.ai_related} for person in extracted_info.people]
    bio_df_new = pd.DataFrame(bio_data)
    print("round: " + counter)
    print(bio_df_new.shape[0])
    # extracted_info_list = pd.concat([extracted_info_list, bio_df_new])
    df_intermediate = pd.concat([df_intermediate, bio_df_new])
df_new = df_new.reset_index(drop=True)
extracted_info_list = df_intermediate.reset_index(drop=True)  # Assuming it's a DataFrame
result = pd.concat([df_new, extracted_info_list], axis=1)
save_excel(result, 'result_temp11.xlsx')